In [9]:
def standings ():

    import pandas as pd
    import numpy as np

    # --------------------

    seventeen = pd.read_csv("../tables/fbref.com/17-18.csv", index_col=[0])
    seventeen["season"] = 17

    eighteen = pd.read_csv("../tables/fbref.com/18-19.csv", index_col=[0])
    eighteen["season"] = 18

    nineteen = pd.read_csv("../tables/fbref.com/19-20.csv", index_col=[0])
    nineteen["season"] = 19

    twenty = pd.read_csv("../tables/fbref.com/20-21.csv", index_col=[0])
    twenty["season"] = 20

    twentyone = pd.read_csv("../tables/fbref.com/21-22.csv", index_col=[0])
    twentyone["season"] = 21

    twentytwo = pd.read_csv("../tables/fbref.com/22-23.csv", index_col=[0])
    twentytwo["season"] = 22

    twentythree = pd.read_csv("../tables/fbref.com/23-24.csv", index_col=[0])
    twentythree["season"] = 23

    season = pd.concat([seventeen, eighteen, nineteen, twenty, twentyone, twentytwo, twentythree])

    # --------------------

    sixteen = pd.read_csv("../tables/football-data.co.uk/16-17.csv")
    sixteen["season"] = 16
    sixteen["matchday"] = np.repeat(np.arange(1, 39), 10)

    fifteen = pd.read_csv("../tables/football-data.co.uk/15-16.csv")
    fifteen["season"] = 15
    fifteen["matchday"] = np.repeat(np.arange(1, 39), 10)

    fourteen = pd.read_csv("../tables/football-data.co.uk/14-15.csv")
    fourteen["season"] = 14
    fourteen["matchday"] = np.repeat(np.arange(1, 39), 10)

    thirteen = pd.read_csv("../tables/football-data.co.uk/14-15.csv")
    thirteen["season"] = 13
    thirteen["matchday"] = np.repeat(np.arange(1, 39), 10)

    twelve = pd.read_csv("../tables/football-data.co.uk/14-15.csv")
    twelve["season"] = 12
    twelve["matchday"] = np.repeat(np.arange(1, 39), 10)

    eleven = pd.read_csv("../tables/football-data.co.uk/14-15.csv")
    eleven["season"] = 11
    eleven["matchday"] = np.repeat(np.arange(1, 39), 10)

    ten = pd.read_csv("../tables/football-data.co.uk/14-15.csv")
    ten["season"] = 10
    ten["matchday"] = np.repeat(np.arange(1, 39), 10)

    footdata = pd.concat([ten, eleven, twelve, thirteen, fourteen, fifteen, sixteen], axis=0)

    footdata = footdata[["season", "matchday", "Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG"]]
    footdata['home_result'] = np.where(footdata['FTHG'] > footdata['FTAG'], 3, np.where(footdata['FTHG'] == footdata['FTAG'], 1, 0))
    footdata['away_result'] = np.where(footdata['FTAG'] > footdata['FTHG'], 3, np.where(footdata['FTAG'] == footdata['FTHG'], 1, 0))
    footdatahome = footdata.drop(["AwayTeam", "FTAG", "FTHG", "away_result"], axis=1).rename(columns={"HomeTeam":"team", "home_result":"result", "Date":"date"})
    footdataaway = footdata.drop(["HomeTeam", "FTAG", "FTHG", "home_result"], axis=1).rename(columns={"AwayTeam":"team", "away_result":"result", "Date":"date"})
    footdata = pd.concat([footdatahome, footdataaway], axis=0).sort_values(by=["season", "matchday", "date"])
    footdata['date'] = pd.to_datetime(footdata['date'], format='%d/%m/%y').dt.date

    # --------------------

    footdata.reset_index(drop=True, inplace=True)

    # --------------------

    season['result'] = np.where(season['score'] > season['score_against'], 3, np.where(season['score'] == season['score_against'], 1, 0))
    season = season[["season", "matchday", "team", "date", "result"]]
    season = pd.concat([footdata, season], axis=0, ignore_index=True)

    season['points'] = season.groupby(["season", "team"])["result"].cumsum()

    season.sort_values(by="season")

    season["rank"] = season.groupby(["season", 'matchday'])['points'].rank("min", ascending=False).astype(int)
    season = season.reset_index().drop(["index"], axis=1)
    season
    season['date'] = pd.to_datetime(season['date'])

    # --------------------

    def win_streak (team):
        streak = season.copy()
        streak = streak[streak["team"]==team]

        streak['win_streak'] = 0

        streak

        target_value = 3

        # Iterate through rows and calculate the streak counter
        current_counter = 0

        for index, row in streak.iterrows():
            if row['result'] == target_value:
                current_counter += 1
            else:
                current_counter = 0

            streak.at[index, 'win_streak'] = current_counter
        return streak
    
    win_streaker = []

    for i in season["team"].unique():
        if len(win_streaker) == 0:
            win_streaker = win_streak(i)
        else:
            win_streaker = pd.concat([win_streaker, win_streak(i)])
    
    def lose_streak (team):
        streak = win_streaker.copy()
        streak = streak[streak["team"]==team]

        streak['lose_streak'] = 0

        streak

        target_value = 0

        # Iterate through rows and calculate the streak counter
        current_counter = 0

        for index, row in streak.iterrows():
            if row['result'] == target_value:
                current_counter += 1
            else:
                current_counter = 0

            streak.at[index, 'lose_streak'] = current_counter
        return streak
    
    lose_streaker = []

    for i in season["team"].unique():
        if len(lose_streaker) == 0:
            lose_streaker = lose_streak(i)
        else:
            lose_streaker = pd.concat([lose_streaker, lose_streak(i)])

    def draw_streak (team):
        streak = lose_streaker.copy()
        streak = streak[streak["team"]==team]

        streak['draw_streak'] = 0

        streak

        target_value = 1

        # Iterate through rows and calculate the streak counter
        current_counter = 0

        for index, row in streak.iterrows():
            if row['result'] == target_value:
                current_counter += 1
            else:
                current_counter = 0

            streak.at[index, 'draw_streak'] = current_counter
        return streak
    
    draw_streaker = []

    for i in season["team"].unique():
        if len(draw_streaker) == 0:
            draw_streaker = draw_streak(i)
        else:
            draw_streaker = pd.concat([draw_streaker, draw_streak(i)])

    draw_streaker.sort_values(by=["season", "matchday", "date"], inplace=True)

    draw_streaker.to_csv("../coding-csv/standings.csv")

In [10]:
standings()